In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime
import time

In [2]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


## Get flight information request

In [11]:
def get_data(airport,status = 'landed'):
    
    KEY = 'b50bc5-5af08a'
    url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={airport}&type=departure&status={status}"    
    response = req.get(url)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()

    if type(json_file) == list:
        df_f = pd.DataFrame(json_file)
        return df_f

    
    
    
    
    

In [12]:
df_barcelona = get_data(airport='BCN')

<Response [200]>
2019-07-25 14:40:33.470605


In [36]:
type(df_barcelona)

pandas.core.frame.DataFrame

In [6]:
df_madrid = get_data(airport='MAD')

<Response [200]>
2019-07-25 14:38:55.853571


TypeError: list indices must be integers or slices, not str

In [15]:
def clean_data(data_df):
    data_df["airln_name"] = data_df["airline"].apply(lambda x: x['name'])
    data_df["airln_code"] = data_df["airline"].apply(lambda x: x['iataCode'])
    data_df['arri_delay'] = data_df['arrival'].apply(lambda x: x['delay'])
    data_df['arri_airpt'] = data_df['arrival'].apply(lambda x: x['iataCode'])
    data_df['schd_arri_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])
    data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)
    data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])
    data_df['depart_airpt'] = data_df['departure'].apply(lambda x: x['iataCode'])
    data_df['schd_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])
    data_df['depart_gate'] = data_df['departure'].apply(lambda x: x['gate'])
    data_df['depart_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])
    data_df['icaonumb'] = data_df['flight'].apply(lambda x: x['icaoNumber'])
    data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])
    data_df = data_df.drop(['airline','arrival','departure'], axis =1)
    
    return data_df
    

In [ ]:
BNC_0 = clean_data(data_df=df_barcelona)

In [ ]:
BNC_0.head()

In [ ]:
BNC_0.to_csv('BNC_0.csv')

In [ ]:
MAD_0 = clean_data(data_df=df_madrid)

In [ ]:
MAD_0.head()

In [ ]:
MAD_0.to_csv('MAD_0.csv')

## Get airports by country

In [16]:
def get_data_airpts(country):
    
    KEY = 'b50bc5-5af08a'
    url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={country}'
    
    response = req.get(url_airpt)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    df = pd.DataFrame(json_file)
    return df

In [17]:
spain_airpts = get_data_airpts('ES')

<Response [200]>
2019-07-25 14:43:19.685737


In [18]:
spain_airpts.head(2)

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,1,27,ABC,ABC,LEAB,ES,6299323,38.948334,-1.863333,Los Llanos,Spain,,Europe/Madrid
1,0,55,ACE,ACE,GCRR,ES,6297189,28.950668,-13.609059,Lanzarote,Spain,902 404 704,Atlantic/Canary


In [19]:
def get_iata_codes(df_airpts):
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    return airpts_lst

In [20]:
spain_airpts_lst = get_iata_codes(df_airpts=spain_airpts)

In [21]:
print(spain_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC']


In [22]:
france_airpts = get_data_airpts('FR')

<Response [200]>
2019-07-25 14:43:26.768482


In [23]:
france_airpts_lst = get_iata_codes(df_airpts=france_airpts)

In [24]:
germany_airpts = get_data_airpts('DE')

<Response [200]>
2019-07-25 14:43:29.140808


In [25]:
germany_airpts_lst = get_iata_codes(df_airpts=germany_airpts)

In [26]:
italy_airpts = get_data_airpts('IT')

<Response [200]>
2019-07-25 14:43:31.721126


In [27]:
italy_airpts_lst = get_iata_codes(df_airpts=italy_airpts)

In [28]:
print(italy_airpts_lst)

['AHO', 'ALL', 'AOI', 'AOT', 'AVB', 'BAU', 'BDS', 'BGY', 'BLQ', 'BLX', 'BRI', 'BRZ', 'BZO', 'BZQ', 'CAG', 'CIA', 'CIY', 'CRV', 'CTA', 'CTJ', 'CUF', 'DCI', 'EBA', 'FCO', 'FLR', 'FNU', 'FOG', 'FOI', 'FRL', 'GBN', 'GOA', 'GRS', 'IBT', 'IMR', 'INP', 'IPR', 'IRR', 'IRT', 'ISH', 'ISR', 'ITT', 'LCC', 'LCV', 'LCZ', 'LIN', 'LMP', 'LTZ', 'MXP', 'NAP', 'NSY', 'OLB', 'PEG', 'PEJ', 'PMF', 'PMO', 'PNL', 'PRJ', 'PSA', 'PSR', 'QSR', 'RAN', 'RCC', 'REG', 'RMI', 'RRO', 'SAY', 'SUF', 'TAR', 'TFC', 'TPS', 'TPY', 'TQR', 'TRN', 'TRS', 'TSF', 'TTB', 'UDN', 'VBS', 'VCE', 'VIC', 'VIF', 'VNZ', 'VRN', 'XIK', 'XIX', 'XNC', 'XRJ', 'XVQ', 'XVY', 'ZAK', 'ZBN', 'ZIE', 'ZIP', 'ZIQ', 'ZJE', 'ZJJ', 'ZJX', 'ZJY', 'ZMF', 'ZMI', 'ZMJ', 'ZMO', 'ZMR', 'ZMS', 'ZRO', 'ZRR', 'ZTC']


In [29]:
countries_airpts_lst = spain_airpts_lst + france_airpts_lst + germany_airpts_lst + italy_airpts_lst

In [44]:
print(countries_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC', 'AGF', 'AHZ', 'AJA', 'ANE', 'ANG', 'AUF', 'AUR', 'AVN', 'BAE', 'BES', 'BIA', 'BIQ', 'BOD', 'BOU', 'BVA', 'BVE', 'BYF', 'BZR', 'CCF', 'CDG', 'CEQ', 'CER', 'CET', 'CFE', 'CFR', 'CHR', 'CLY', 'CMF', 'CMR', 'CNG', 'CQF', 'CSF', 'CTT', 'CVF', 'DCM', 'DIJ', 'DLE', 'DNR', 'DOL'

In [40]:
df_countries_airpt = pd.DataFrame(countries_airpts_lst)

## Wednesday, July 24

In [39]:
dict_airpt_df = {}

for aipt_code in countries_airpts_lst:
    
    df_airpt = get_data(airport = f"{aipt_code}")
    
    if type(df_airpt) == pd.core.frame.DataFrame:
        df_airpt_clean = clean_data(df_airpt)
        dict_airpt_df[aipt_code] = df_airpt_clean
        time.sleep(10)


<Response [200]>
2019-07-25 14:49:00.948145
<Response [200]>
2019-07-25 14:49:02.018871
<Response [200]>
2019-07-25 14:49:13.026360
<Response [200]>
2019-07-25 14:49:14.194055
<Response [200]>
2019-07-25 14:49:24.871619
<Response [200]>
2019-07-25 14:49:35.914823
<Response [200]>
2019-07-25 14:49:46.547661
<Response [200]>
2019-07-25 14:49:57.302341
<Response [200]>
2019-07-25 14:50:08.287341
<Response [200]>
2019-07-25 14:50:08.943086
<Response [200]>
2019-07-25 14:50:09.807562
<Response [200]>
2019-07-25 14:50:20.552182
<Response [200]>
2019-07-25 14:50:21.351527
<Response [200]>
2019-07-25 14:50:22.159588
<Response [200]>
2019-07-25 14:50:22.880731
<Response [200]>
2019-07-25 14:50:23.555918
<Response [200]>
2019-07-25 14:50:24.452394
<Response [200]>
2019-07-25 14:50:25.251456
<Response [200]>
2019-07-25 14:50:36.099010
<Response [200]>
2019-07-25 14:50:36.933564
<Response [200]>
2019-07-25 14:50:37.802731
<Response [200]>
2019-07-25 14:50:48.887133
<Response [200]>
2019-07-25 14:5

<Response [200]>
2019-07-25 15:00:42.002403
<Response [200]>
2019-07-25 15:00:42.945028
<Response [200]>
2019-07-25 15:00:53.435532
<Response [200]>
2019-07-25 15:00:54.001303
<Response [200]>
2019-07-25 15:00:54.720477
<Response [200]>
2019-07-25 15:01:05.243067
<Response [200]>
2019-07-25 15:01:05.728944
<Response [200]>
2019-07-25 15:01:06.209153
<Response [200]>
2019-07-25 15:01:07.126361
<Response [200]>
2019-07-25 15:01:17.834827
<Response [200]>
2019-07-25 15:01:28.382977
<Response [200]>
2019-07-25 15:01:29.115046
<Response [200]>
2019-07-25 15:01:39.588849
<Response [200]>
2019-07-25 15:01:40.075001
<Response [200]>
2019-07-25 15:01:40.675018
<Response [200]>
2019-07-25 15:01:51.180892
<Response [200]>
2019-07-25 15:01:51.817982
<Response [200]>
2019-07-25 15:01:52.379705
<Response [200]>
2019-07-25 15:01:52.921148
<Response [200]>
2019-07-25 15:01:53.401118
<Response [200]>
2019-07-25 15:01:54.004434
<Response [200]>
2019-07-25 15:01:54.761051
<Response [200]>
2019-07-25 15:0

<Response [200]>
2019-07-25 15:08:36.432179
<Response [200]>
2019-07-25 15:08:37.090457
<Response [200]>
2019-07-25 15:08:37.727617
<Response [200]>
2019-07-25 15:08:38.317605
<Response [200]>
2019-07-25 15:08:39.129945
<Response [200]>
2019-07-25 15:08:40.110344
<Response [200]>
2019-07-25 15:08:40.985091
<Response [200]>
2019-07-25 15:08:42.003609
<Response [200]>
2019-07-25 15:08:42.831991
<Response [200]>
2019-07-25 15:08:43.520458
<Response [200]>
2019-07-25 15:08:44.239167
<Response [200]>
2019-07-25 15:08:44.919526
<Response [200]>
2019-07-25 15:08:45.732804
<Response [200]>
2019-07-25 15:08:46.815166
<Response [200]>
2019-07-25 15:08:47.728667
<Response [200]>
2019-07-25 15:08:48.534725
<Response [200]>
2019-07-25 15:08:49.233713
<Response [200]>
2019-07-25 15:08:49.846726
<Response [200]>
2019-07-25 15:08:50.767540
<Response [200]>
2019-07-25 15:08:51.651994
<Response [200]>
2019-07-25 15:08:52.434677
<Response [200]>
2019-07-25 15:08:53.162556
<Response [200]>
2019-07-25 15:0

In [47]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [48]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


In [53]:
def save_to_csv():
    for key, value in dict_airpt_df.items():
        
        value = value.astype({'schd_arri_time': 'datetime64[ns]','schd_depart_time':'datetime64[ns]'})
        
        value = value[['airln_name', 'airln_code','type', 'codeshared', 'status', 'icaonumb', 'flight','depart_terminal',\
      'depart_gate', 'depart_airpt' ,'schd_depart_time','depart_delay','arri_airpt','schd_arri_time',\
      'arri_delay']]
        value['depart_delay'] = value['depart_delay'].fillna(0)
        value['arri_delay'] = value['arri_delay'].fillna(0)
        
        value.to_sql('flights',engine, if_exists='append', index=False)

In [54]:
save_to_csv()